In [1]:
import pandas as pd
import numpy as np
from io import StringIO
from sqlalchemy import create_engine

In [2]:
basedata_df = pd.read_csv("shark_attacks.csv")

In [3]:
basedata_df = basedata_df.drop(columns = ["Unnamed: 17", "Unnamed: 18"])
basedata_df.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,href formula
0,2018.06.25,25-Jun-18,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",http://sharkattackfile.net/spreadsheets/pdf_di...
1,2018.06.18,18-Jun-18,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",http://sharkattackfile.net/spreadsheets/pdf_di...
2,2018.06.09,9-Jun-18,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",http://sharkattackfile.net/spreadsheets/pdf_di...
3,2018.06.08,8-Jun-18,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",http://sharkattackfile.net/spreadsheets/pdf_di...
4,2018.06.04,4-Jun-18,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,http://sharkattackfile.net/spreadsheets/pdf_di...


In [4]:
df_cleaning = basedata_df.dropna()

In [5]:
basedata_df.describe()

,Year
count,6300.000000
mean,1927.272381
std,281.116308
min,0.000000
25%,1942.000000
50%,1977.000000
75%,2005.000000
max,2018.000000


In [6]:
df_cleaning.describe()

,Year
count,1423.000000
mean,1987.446943
std,94.361381
min,0.000000
25%,1978.000000
50%,2000.000000
75%,2009.000000
max,2018.000000


In [7]:
df_cleaning["Year"].min()

0.0

In [8]:
df_cleaning = df_cleaning[df_cleaning["Year"] != 0]

In [9]:
df_cleaning["Year"].min()

1868.0

In [10]:
df_cleaning["Year"].max()

2018.0

In [11]:
list(df_cleaning)

['Case Number',
 'Date',
 'Year',
 'Type',
 'Country',
 'Area',
 'Location',
 'Activity',
 'Name',
 'Sex',
 'Age',
 'Injury',
 'Fatal (Y/N)',
 'Time',
 'Species',
 'Investigator or Source',
 'href formula']

In [12]:
df_cleaning.Sex.unique()

array(['F', 'M', 'lli'], dtype=object)

In [13]:
df_cleaning = df_cleaning[df_cleaning["Sex"] != 'lli']
df_cleaning.Sex.unique()

array(['F', 'M'], dtype=object)

In [14]:
df_cleaning.Type.unique()

array(['Boating', 'Unprovoked', 'Provoked', 'Invalid', 'Sea Disaster'],
      dtype=object)

In [15]:
# strip all leading and trailing spaces of Country column
df_cleaning.Country = df_cleaning.Country.str.strip()
df_cleaning.Country.unique()

array(['USA', 'BRAZIL', 'AUSTRALIA', 'ENGLAND', 'COSTA RICA', 'MALDIVES',
       'SOUTH AFRICA', 'CUBA', 'BAHAMAS', 'INDONESIA', 'SPAIN', 'EGYPT',
       'NEW ZEALAND', 'FRENCH POLYNESIA', 'NEW CALEDONIA', 'ECUADOR',
       'THAILAND', 'REUNION', 'MEXICO', 'JAMAICA', 'ITALY',
       'PAPUA NEW GUINEA', 'RUSSIA', 'SEYCHELLES', 'VIETNAM',
       'MOZAMBIQUE', 'SCOTLAND', 'UNITED KINGDOM', 'TAIWAN', 'GUAM',
       'CROATIA', 'FIJI', 'TONGA', 'ST. MAARTIN', 'GRAND CAYMAN',
       'VANUATU', 'VENEZUELA', 'TURKS & CAICOS', 'OKINAWA',
       'MARSHALL ISLANDS', 'HONG KONG', 'JAPAN', 'SOUTH KOREA', 'ISRAEL',
       'MICRONESIA', 'COLUMBIA', 'NEW BRITAIN', 'PALAU',
       'SOLOMON ISLANDS', 'FEDERATED STATES OF MICRONESIA', 'GREECE',
       'BRITISH WEST INDIES', 'NORTH PACIFIC OCEAN', 'BERMUDA',
       'AMERICAN SAMOA', 'PACIFIC OCEAN', 'SENEGAL', 'ARGENTINA',
       'EL SALVADOR', 'IRAQ', 'PANAMA', 'INDIA'], dtype=object)

In [16]:
# strip all leading and trailing spaces of Area column
df_cleaning.Area = df_cleaning.Area.str.strip()
df_cleaning.Area.unique()

array(['California', 'Pernambuco', 'Florida', 'Queensland', 'Cornwall',
       'Cocos Island', 'Western Australia', 'Alifu Alifu Atoll',
       'Western Cape Province', 'Eastern Cape Province', 'Hawaii',
       'Victoria', 'New South Wales', 'Fernando de Noronha',
       'Holquin Province', 'South Australia', 'Massachusetts',
       'South Carolina', 'New Providence', 'Bali', 'Alicante Province',
       'Texas', 'North Carolina', 'Suez', 'North Island', 'Tuamotos',
       'Grand Terre', 'Galapagos Islands', 'Phuket', 'Tasmania',
       'Le Port', 'Rangiroa', 'Saint-Gilles-les-Bains', 'Bora Bora',
       'd’Étang-Salé', 'Abaco Islands', 'Alabama', 'Louisiana',
       'Delaware', 'Society Islands', 'South Island', 'North Province',
       'Oregon', 'Exuma Islands', 'Santa Cruz Island', 'Saint-Gilles',
       'Quintana Roo', 'Tuamotus', 'St. Catherine', 'Sinaloa', 'Sardinia',
       'Guerrero', 'Central Province',
       'Telyakovsky Bay, Khasan,  Primorsky Krai (Far East)', 'Praslin',
  

In [17]:
df_cleaning = df_cleaning[df_cleaning["Area"] != '0ºS, 142ºE']
df_cleaning = df_cleaning[df_cleaning["Area"] != 'Vava\x92u']
df_cleaning.Area.unique()

array(['California', 'Pernambuco', 'Florida', 'Queensland', 'Cornwall',
       'Cocos Island', 'Western Australia', 'Alifu Alifu Atoll',
       'Western Cape Province', 'Eastern Cape Province', 'Hawaii',
       'Victoria', 'New South Wales', 'Fernando de Noronha',
       'Holquin Province', 'South Australia', 'Massachusetts',
       'South Carolina', 'New Providence', 'Bali', 'Alicante Province',
       'Texas', 'North Carolina', 'Suez', 'North Island', 'Tuamotos',
       'Grand Terre', 'Galapagos Islands', 'Phuket', 'Tasmania',
       'Le Port', 'Rangiroa', 'Saint-Gilles-les-Bains', 'Bora Bora',
       'd’Étang-Salé', 'Abaco Islands', 'Alabama', 'Louisiana',
       'Delaware', 'Society Islands', 'South Island', 'North Province',
       'Oregon', 'Exuma Islands', 'Santa Cruz Island', 'Saint-Gilles',
       'Quintana Roo', 'Tuamotus', 'St. Catherine', 'Sinaloa', 'Sardinia',
       'Guerrero', 'Central Province',
       'Telyakovsky Bay, Khasan,  Primorsky Krai (Far East)', 'Praslin',
  

In [18]:
# strip all leading and trailing spaces of Location column
df_cleaning.Location = df_cleaning.Location.str.strip()
df_cleaning.Location.unique()

array(['Oceanside, San Diego County', 'Piedade Beach, Recife',
       'Cocoa Beach, Brevard  County', ..., 'Lake Macquarie',
       'Petrie Bight, Brisbane River', 'Ghat'], dtype=object)

In [19]:
# strip all leading and trailing spaces of Fatal column
df_cleaning["Fatal (Y/N)"] = df_cleaning["Fatal (Y/N)"].str.strip()
df_cleaning["Fatal (Y/N)"].unique()

array(['N', 'Y', 'M', '2017'], dtype=object)

In [20]:
# Drop rows with Fatal column values of 'M' and '2017'
df_cleaning = df_cleaning[df_cleaning["Fatal (Y/N)"] != 'M']
df_cleaning = df_cleaning[df_cleaning["Fatal (Y/N)"] != '2017']
df_cleaning["Fatal (Y/N)"].unique()

array(['N', 'Y'], dtype=object)

In [21]:
df_cleaning.describe()

,Year
count,1417.000000
mean,1991.605505
std,23.553558
min,1868.000000
25%,1979.000000
50%,2000.000000
75%,2009.000000
max,2018.000000


In [22]:
# Drop rows in Age column that are not numeric
df_cleaning = df_cleaning[df_cleaning.Age.apply(lambda x: x.isnumeric())]
df_cleaning.Age.str.isnumeric().unique()

array([ True])

In [23]:
df_cleaning.Age.unique()

array(['57', '18', '15', '32', '21', '30', '60', '33', '19', '25', '10',
       '69', '55', '35', '20', '54', '22', '31', '40', '37', '13', '34',
       '50', '46', '48', '17', '28', '65', '73', '58', '36', '51', '61',
       '59', '42', '6', '23', '29', '39', '24', '12', '26', '11', '71',
       '43', '44', '14', '27', '62', '52', '38', '68', '16', '47', '63',
       '70', '9', '41', '53', '7', '66', '45', '74', '64', '8', '56',
       '49', '77', '84', '6½', '5'], dtype=object)

In [24]:
df_cleaning = df_cleaning[df_cleaning["Age"] != '6½']
df_cleaning.Age.unique()

array(['57', '18', '15', '32', '21', '30', '60', '33', '19', '25', '10',
       '69', '55', '35', '20', '54', '22', '31', '40', '37', '13', '34',
       '50', '46', '48', '17', '28', '65', '73', '58', '36', '51', '61',
       '59', '42', '6', '23', '29', '39', '24', '12', '26', '11', '71',
       '43', '44', '14', '27', '62', '52', '38', '68', '16', '47', '63',
       '70', '9', '41', '53', '7', '66', '45', '74', '64', '8', '56',
       '49', '77', '84', '5'], dtype=object)

In [25]:
df_cleaning.describe()

,Year
count,1399.000000
mean,1991.598999
std,23.548174
min,1868.000000
25%,1979.000000
50%,2000.000000
75%,2009.000000
max,2018.000000


In [26]:
# strip all leading and trailing spaces of Injury column
df_cleaning.Injury = df_cleaning.Injury.str.strip()
df_cleaning.Injury.unique()

array(['No injury to occupant, outrigger canoe and paddle damaged',
       'FATAL', 'Lower left leg bitten',
       'Minor bite to hand by captive shark. PROVOKED INCIDENT',
       'Injured by teeth of a dead porbeagle shark he was tossing overboard.',
       'No injury, shark bit scuba gear',
       'No injury, knocked off board by shark', '5-inch cut to hand',
       'No injury, shark bit hole in ski',
       'Lacerations to left knee & lower leg',
       'Injuries to right leg & hand', 'Minor injuries to legs',
       'No injury, said to have been charged by sharks several times',
       'Minor injury, ankle grazed', 'Laceratons to right leg & foot',
       'Puncture wounds to leff foot & lower leg',
       'Minor injury to left forearm',
       'Severe lacerations to left thigh & knee', 'Right leg bitten',
       'Puncture wounds to left shoulder', 'Minor injury to foot',
       'Minor injuries', 'Lacerations to left hand', 'Abrasions',
       'Lacerations to left foot', 'Minor lac

In [27]:
df_cleaning.count()

Case Number               1399
Date                      1399
Year                      1399
Type                      1399
Country                   1399
Area                      1399
Location                  1399
Activity                  1399
Name                      1399
Sex                       1399
Age                       1399
Injury                    1399
Fatal (Y/N)               1399
Time                      1399
Species                   1399
Investigator or Source    1399
href formula              1399
dtype: int64

In [28]:
# strip all leading and trailing spaces of Activity column
df_cleaning.Activity = df_cleaning.Activity.str.strip()
df_cleaning.Activity.unique()

array(['Paddling', 'Swimming', 'Walking', 'Feeding sharks', 'Fishing',
       'Scuba diving', 'Surfing', 'Paddle-skiing',
       'Stand-Up Paddleboarding', 'Wading', 'Spearfishing',
       'Night bathing', 'Kayaking / Fishing', 'Snorkeling', 'SUP',
       'Kayaking', 'Body boarding', 'Scuba Diving', 'Surf skiing',
       'Surf fishing', 'Floating', 'SUP Foil boarding', 'Kite surfing',
       'Floating in tube', 'Standing', 'Diving', 'Teasing a shark',
       'Paddle boarding', 'Kayak Fishing', 'Surf-skiing',
       'Scallop diving on hookah', 'Playing in the water',
       'Body Boarding', 'Boogie boarding', 'Playing',
       'Standing / Snorkeling', 'Hand feeding sharks', 'Rowing',
       'Shark fishing', 'Body surfing', 'Kitesurfing', 'Kiteboarding',
       'Swimming / snorkeling', 'Diving for Abalone', 'Casting a net',
       'Cleaning fish', 'Boogie Boarding', 'Playing in the surf',
       'Kite boarding', "Fishing - 'tag & release'", 'Bodyboarding',
       'Swimming or boogie boar

In [29]:
df_cleaning.Activity.nunique()

323

In [30]:
df_cleaning.loc[df_cleaning['Activity'].str.contains('fishing', case=False), 'Activity'] = 'Fishing'
df_cleaning.Activity.nunique()

269

In [31]:
df_cleaning.loc[df_cleaning['Activity'].str.contains('scuba', case=False), 'Activity'] = 'Scuba Diving'
df_cleaning.Activity.nunique()

255

In [32]:
df_cleaning.loc[df_cleaning['Activity'].str.contains('surfing', case=False), 'Activity'] = 'Surfing'
df_cleaning.loc[df_cleaning['Activity'].str.contains('surfboard', case=False), 'Activity'] = 'Surfing'
df_cleaning.Activity.nunique()

213

In [33]:
df_cleaning.loc[df_cleaning['Activity'].str.contains('swimming', case=False), 'Activity'] = 'Swimming'
df_cleaning.Activity.nunique()

173

In [34]:
df_cleaning.loc[df_cleaning['Activity'].str.contains('Foil boarding', case=False), 'Activity'] = 'Foil boarding'
df_cleaning.Activity.nunique()

173

In [35]:
df_cleaning.loc[df_cleaning['Activity'].str.contains('paddle boarding', case=False), 'Activity'] = 'Paddle Boarding'
df_cleaning.loc[df_cleaning['Activity'].str.contains('Stand-Up Paddleboarding', case=False), 'Activity'] = 'Paddle Boarding'
df_cleaning.loc[df_cleaning['Activity'].str.contains('SUP'), 'Activity'] = 'Paddle Boarding'
df_cleaning.Activity.nunique()

170

In [36]:
df_cleaning.loc[df_cleaning['Activity'].str.contains('bathing', case=False), 'Activity'] = 'Bathing'
df_cleaning.Activity.nunique()

165

In [37]:
df_cleaning.loc[df_cleaning['Activity'].str.contains('snorkeling', case=False), 'Activity'] = 'Snorkeling'
df_cleaning.Activity.nunique()

161

In [38]:
df_cleaning.loc[df_cleaning['Activity'].str.contains('floating', case=False), 'Activity'] = 'Floating'
df_cleaning.loc[df_cleaning['Activity'].str.contains('floatation', case=False), 'Activity'] = 'Floating'
df_cleaning.Activity.nunique()

154

In [39]:
df_cleaning.loc[df_cleaning['Activity'].str.contains('splashing', case=False), 'Activity'] = 'Splashing'
df_cleaning.Activity.nunique()

152

In [40]:
df_cleaning.loc[df_cleaning['Activity'].str.contains('skindiving', case=False), 'Activity'] = 'Skindiving'
df_cleaning.loc[df_cleaning['Activity'].str.contains('Skin diving', case=False), 'Activity'] = 'Skindiving'
df_cleaning.Activity.nunique()

149

In [41]:
df_cleaning.loc[df_cleaning['Activity'].str.contains('abalone', case=False), 'Activity'] = 'Abalone Diving'
df_cleaning.Activity.nunique()

139

In [42]:
df_cleaning.loc[df_cleaning['Activity'].str.contains('Paddleskiing', case=False), 'Activity'] = 'Paddle-skiing'
df_cleaning.Activity.nunique()

138

In [43]:
df_cleaning.loc[df_cleaning['Activity'].str.contains('Surf skiing', case=False), 'Activity'] = 'Surf-skiing'
df_cleaning.Activity.nunique()

137

In [44]:
df_cleaning.loc[df_cleaning['Activity'].str.contains('Bodyboarding', case=False), 'Activity'] = 'Body-boarding'
df_cleaning.loc[df_cleaning['Activity'].str.contains('Body Boarding', case=False), 'Activity'] = 'Body-boarding'
df_cleaning.Activity.nunique()

134

In [45]:
df_cleaning.Activity.unique()

array(['Paddling', 'Swimming', 'Walking', 'Feeding sharks', 'Fishing',
       'Scuba Diving', 'Surfing', 'Paddle-skiing', 'Paddle Boarding',
       'Wading', 'Bathing', 'Snorkeling', 'Kayaking', 'Body-boarding',
       'Surf-skiing', 'Floating', 'Foil boarding', 'Standing', 'Diving',
       'Teasing a shark', 'Scallop diving on hookah',
       'Playing in the water', 'Boogie boarding', 'Playing',
       'Hand feeding sharks', 'Rowing', 'Kiteboarding', 'Abalone Diving',
       'Casting a net', 'Cleaning fish', 'Boogie Boarding',
       'Playing in the surf', 'Kite boarding', 'Free diving',
       'Sitting in the water', 'Diving, feeding sharks', 'Kite Boarding',
       'Washing sand off a speared fish', 'Wakeboarding', 'Rescuing',
       'Measuring sharks',
       'Diving, but on the surface when bitten by the shark',
       'Treading water', 'Jet skiing',
       "Accidentally stood on hooked shark's tail before attempting to gut it",
       'Jumping', 'Jumped into the water', 'Reviving

In [56]:
df = df_cleaning.rename(columns={"Case Number": "Case_Number", "Fatal (Y/N)": "Fatal_Y_N", "href formula": "href_formula"})
df.head()

,Case_Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal_Y_N,Time,Species,Investigator or Source,href_formula
0,2018.06.25,25-Jun-18,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",http://sharkattackfile.net/spreadsheets/pdf_di...
6,2018.06.03.a,3-Jun-18,2018.0,Unprovoked,BRAZIL,Pernambuco,"Piedade Beach, Recife",Swimming,Jose Ernesto da Silva,M,18,FATAL,Y,Late afternoon,Tiger shark,"Diario de Pernambuco, 6/4/2018",http://sharkattackfile.net/spreadsheets/pdf_di...
8,2018.05.26.b,26-May-18,2018.0,Unprovoked,USA,Florida,"Cocoa Beach, Brevard County",Walking,Cody High,M,15,Lower left leg bitten,N,17h00,"Bull shark, 6'","K.McMurray, TrackingSharks.com",http://sharkattackfile.net/spreadsheets/pdf_di...
10,2018.05.24,24-May-18,2018.0,Provoked,AUSTRALIA,Queensland,Cairns Aquarium,Feeding sharks,male,M,32,Minor bite to hand by captive shark. PROVOKED ...,N,Morning,Grey reef shark,"ABC.net.au ,05/24/2018",http://sharkattackfile.net/spreadsheets/pdf_di...
13,2018.05.13.a,13-May-18,2018.0,Invalid,ENGLAND,Cornwall,Off Land's End,Fishing,Max Berryman,M,21,Injured by teeth of a dead porbeagle shark he ...,N,08h15,Invalid incident,"K. McMurray, TrackingSharks.com",http://sharkattackfile.net/spreadsheets/pdf_di...


In [57]:
df.to_csv("shark_data_cleaned.csv", index=False)

## Export to Database

In [58]:
engine = create_engine('sqlite:///shark.sqlite', echo=False)

In [59]:
df.to_sql('shark_data_cleaned', con=engine, if_exists='replace', index_label='id')
engine.execute("SELECT * FROM shark_data_cleaned").fetchall()

[(0, '2018.06.25', '25-Jun-18', 2018.0, 'Boating', 'USA', 'California', 'Oceanside, San Diego County', 'Paddling', 'Julie Wolfe', 'F', '57', 'No injury to occupant, outrigger canoe and paddle damaged', 'N', '18h00', 'White shark', 'R. Collier, GSAF', 'http://sharkattackfile.net/spreadsheets/pdf_directory/2018.06.25-Wolfe.pdf'),
 (6, '2018.06.03.a', '3-Jun-18', 2018.0, 'Unprovoked', 'BRAZIL', 'Pernambuco', 'Piedade Beach, Recife', 'Swimming', 'Jose Ernesto da Silva ', 'M', '18', 'FATAL', 'Y', 'Late afternoon', 'Tiger shark', 'Diario de Pernambuco, 6/4/2018', 'http://sharkattackfile.net/spreadsheets/pdf_directory/2018.06.03.a-daSilva.pdf'),
 (8, '2018.05.26.b', '26-May-18', 2018.0, 'Unprovoked', 'USA', 'Florida', 'Cocoa Beach, Brevard  County', 'Walking', 'Cody High', 'M', '15', 'Lower left leg bitten', 'N', '17h00', "Bull shark, 6'", 'K.McMurray, TrackingSharks.com', 'http://sharkattackfile.net/spreadsheets/pdf_directory/2018.05.26.b-High.pdf'),
 (10, '2018.05.24', '24-May-18', 2018.0, 

Test Visuals


In [70]:
fatal_df = df.loc[df['Fatal_Y_N'] == "Y"].count()
fatal_df


Case_Number               228
Date                      228
Year                      228
Type                      228
Country                   228
Area                      228
Location                  228
Activity                  228
Name                      228
Sex                       228
Age                       228
Injury                    228
Fatal_Y_N                 228
Time                      228
Species                   228
Investigator or Source    228
href_formula              228
dtype: int64

In [98]:
activity_df = df.(Ac)
activity_dict


{'Case_Number': Activity
 Paddling                         2018.06.25
 Swimming                       2018.06.03.a
 Walking                        2018.05.26.b
 Feeding sharks                   2018.05.24
 Fishing                        2018.05.13.a
 Scuba Diving                   2018.04.28.b
 Surfing                        2018.04.25.a
 Fishing                          2018.04.23
 Paddle-skiing                    2018.04.22
 Surfing                          2018.04.03
 Paddle Boarding                  2018.03.31
 Wading                           2018.03.14
 Swimming                       2018.03.09.b
 Surfing                        2018.03.09.a
 Swimming                         2018.02.23
 Walking                          2018.02.15
 Surfing                          2018.01.12
 Surfing                          2017.12.31
 Fishing                          2017.11.24
 Surfing                          2017.11.13
 Bathing                          2017.11.04
 Surfing                      